# Library

In [ ]:
import os
import numpy as np
import pandas as pd
import Levenshtein
import cv2
from PIL import Image
from tqdm.auto import tqdm

# Data Loading

In [ ]:
%%time

train = pd.read_csv('../input/bms-molecular-translation/train_labels.csv')
test = pd.read_csv('../input/bms-molecular-translation/sample_submission.csv')

def get_train_file_path(image_id):
    return "../input/bms-molecular-translation/train/{}/{}/{}/{}.png".format(
        image_id[0], image_id[1], image_id[2], image_id 
    )

def get_test_file_path(image_id):
    return "../input/bms-molecular-translation/test/{}/{}/{}/{}.png".format(
        image_id[0], image_id[1], image_id[2], image_id 
    )

# Scoring function

In [ ]:
def get_score(y_true, y_pred):
    scores = []
    for true, pred in zip(y_true, y_pred):
        score = Levenshtein.distance(true, pred)
        scores.append(score)
    avg_score = np.mean(scores)
    return avg_score

# Naive baseline

In [ ]:
df = train
c = ''
for l in range(16):
    c += df['InChI'].str[l:l+1].mode()[0]
df = train[train['InChI'].str.contains(c)]
print(c)

In [ ]:
df.shape[0]

In [ ]:
m = 999
cm = ''
y_true = df['InChI'].values
for c in tqdm(y_true):
    score = get_score(y_true, [c] * len(df))
    if score<m:
        m = score
        cm = c
print(m, cm)

In [ ]:
y_true = train['InChI'].values
score = get_score(y_true, [cm] * len(train))
print(score)

# Submission

In [ ]:
test['InChI'] = cm
output_cols = ['image_id', 'InChI']
display(test[output_cols])
test[output_cols].to_csv('submission.csv.gz', compression='gzip', index=False)